In [1]:
import sys
!{sys.executable} -m pip install sklearn

In [4]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

In [5]:
def one_hot(i):
    a = [0,0,0,0]
    a[i-769] = 1;
    return a

In [74]:
# Load subject data and store the reshaped data in X_sub[i], y_sub[i] 
X_sub = []
y_sub = []
for i in np.arange(9):
    data = h5py.File('project_datasets/A0'+str(i+1)+'T_slice.mat', 'r')
    X_sub.append(np.copy(data['image']))
    y_sub.append(np.copy(data['type']))
    X_sub[i] = X_sub[i][:, :22, :]
    X_sub[i] = X_sub[i].transpose([0,2,1])
    y_sub[i] = y_sub[i][0,0:X_sub[i].shape[0]:1]
    y_sub[i] = np.asarray(y_sub[i], dtype=np.int32)
    y_sub[i] = [one_hot(j) for j in y_sub[i]]
    y_sub[i] = np.asarray(y_sub[i], dtype = np.int8)

In [75]:
# Remove NaN
for i in np.arange(9):
    y_sub[i] = y_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]
    X_sub[i] = X_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]

In [169]:
# Data Normalization
def normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/ sigma

In [319]:
# For training across all subjects
# Shuffle the data, sample 20% for testing, and the rest for training
X = []
y = []

X = np.concatenate([X_sub[i] for i in np.arange(9)])
y = np.concatenate([y_sub[i] for i in np.arange(9)])
num_trial = X.shape[0]
index_shuffle = np.arange(num_trial)
np.random.shuffle(index_shuffle)
X_shuffle = X[index_shuffle]
y_shuffle = y[index_shuffle]

training_index = int(np.floor(num_trial * (1 - 0.2)))
#validation_index = training_index + int(np.floor(num_trial * 0.2))
# test_index = num_trial

X_train = X_shuffle[0:training_index]
y_train = y_shuffle[0:training_index]
# X_val = X_shuffle[training_index:validation_index]
# y_val = y_shuffle[training_index:validation_index]
# y_val_steps = np.tile(y_val,((X_train.shape[1]),1))
X_test = X_shuffle[training_index:num_trial]
y_test = y_shuffle[training_index:num_trial]
y_test_steps = np.tile(y_test,((X_train.shape[1]),1))

X_train = normalize(X_train)
#X_val = normalize(X_val)
X_test = normalize(X_test)

print(X_train.shape)
print(X_test.shape)
print(y_test_steps.shape)

(2046, 1000, 22)
(512, 1000, 22)
(512000, 4)


In [220]:
# For training within each subject[i]
# Shuffle the data, sample 50 trials for testing, and the rest for training -->
X_train_sub = []
y_train_sub = []
X_test_sub = []
y_test_sub = []
y_test_steps_sub = []

for i in np.arange(9):
    num_trial_sub = X_sub[i].shape[0]
    index_shuffle_sub = np.arange(num_trial_sub)
    np.random.shuffle(index_shuffle_sub)
    X_shuffle_sub = X_sub[i][index_shuffle_sub]
    y_shuffle_sub = y_sub[i][index_shuffle_sub]
    
    training_index_sub = num_trial_sub - 50
    
    X_train_sub.append(X_shuffle_sub[0:training_index_sub])
    y_train_sub.append(y_shuffle_sub[0:training_index_sub])
    X_test_sub.append(X_shuffle[training_index_sub:num_trial_sub])
    y_test_sub.append(y_shuffle[training_index_sub:num_trial_sub])
    y_test_steps_sub.append(np.tile(y_test_sub[i],((X_train_sub[i].shape[1]),1)))

    X_train_sub[i] = normalize(X_train_sub[i])
    X_test_sub[i] = normalize(X_test_sub[i])

    print(X_train_sub[i].shape)
    print(X_test_sub[i].shape)
    print(y_test_steps_sub[i].shape)    

(237, 1000, 22)
(50, 1000, 22)
(50000, 4)
(236, 1000, 22)
(50, 1000, 22)
(50000, 4)
(236, 1000, 22)
(50, 1000, 22)
(50000, 4)
(234, 1000, 22)
(50, 1000, 22)
(50000, 4)
(232, 1000, 22)
(50, 1000, 22)
(50000, 4)
(235, 1000, 22)
(50, 1000, 22)
(50000, 4)
(238, 1000, 22)
(50, 1000, 22)
(50000, 4)
(232, 1000, 22)
(50, 1000, 22)
(50000, 4)
(228, 1000, 22)
(50, 1000, 22)
(50000, 4)


In [320]:
# For training across all subjects: Hyperparameters setting
tf.reset_default_graph()

learning_rate = 0.0005
training_epochs = 30
batch_size = 35
total_batches = (X_train.shape[0]//batch_size)
# print(total_batches)

alpha = 0.3
reg = 1

In [309]:
# For training within each subject: Hyperparameters setting
tf.reset_default_graph()

learning_rate_sub = 0.001
training_epochs_sub = 10
batch_size_sub = 15
total_batches_sub = (X_train_sub[i].shape[0]//batch_size_sub)
# print(total_batches)

alpha_sub = 0.3
reg_sub = 1

In [321]:
# Fixed Parameters
n_input = 22
n_steps = 1000
n_hidden = 128
n_classes = 4

# Common Parameters
n_layers = 2

# Placeholders
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])
dropout = tf.placeholder(tf.float32) # prob to drop the cell

In [322]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cells = []
    for i in np.arange(n_layers):
        cell = rnn_cell.LSTMCell(n_hidden,state_is_tuple = True)
        cell = tf.contrib.rnn.DropoutWrapper(
            cell, output_keep_prob=1.0 - dropout)
        cells.append(cell)
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    output, state = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1)  

    return output_last, output_all

In [323]:
weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
y_last, y_all = LSTM(x,weight,bias)

In [324]:
all_steps_cost = -tf.reduce_mean((y_steps * tf.log(y_all))  + (1 - y_steps) * tf.log(1 - y_all))
last_step_cost = -tf.reduce_mean((y * tf.log(y_last)) + ((1 - y) * tf.log(1 - y_last)))
regularizer = tf.nn.l2_loss(weight)
loss_function = (alpha * all_steps_cost) + ((1 - alpha) * last_step_cost) + reg * 0.5 *  regularizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(y_last, axis = 1),tf.argmax(y,axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

/Users/SoniaYu/miniconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# Training across all subjects

display_step = 20 

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (y_train.shape[0] - batch_size)
            batch_x = X_train[offset:(offset + batch_size), :]
            batch_y = y_train[offset:(offset + batch_size), :]
            batch_y_steps = np.tile(batch_y,((X_train.shape[1]),1))
            
            _, acc, loss = session.run([optimizer, accuracy, loss_function], feed_dict={
                x: batch_x, y: batch_y, y_steps: batch_y_steps, dropout: 0.3})
            
            if (b % display_step == 0):
                print('training accuracy = %.2f, training loss = %f' % (acc, loss))
            
        test_accuracy = session.run(accuracy, feed_dict={
            x:X_test, y:y_test, y_steps: y_test_steps, dropout: 0})
        print('Epoch %d: test accuracy = %f' % (epoch, test_accuracy))
            
    

training accuracy = 0.23, training loss = 1.677920
training accuracy = 0.11, training loss = 1.384351
training accuracy = 0.20, training loss = 1.241838
Epoch 0: test accuracy = 0.236328
training accuracy = 0.20, training loss = 1.137630
training accuracy = 0.23, training loss = 1.031423
training accuracy = 0.14, training loss = 0.963401
Epoch 1: test accuracy = 0.240234
training accuracy = 0.09, training loss = 0.897811
training accuracy = 0.34, training loss = 0.825724
training accuracy = 0.06, training loss = 0.797855
Epoch 2: test accuracy = 0.236328
training accuracy = 0.20, training loss = 0.753752
training accuracy = 0.31, training loss = 0.713269
training accuracy = 0.17, training loss = 0.695013
Epoch 3: test accuracy = 0.232422
training accuracy = 0.17, training loss = 0.676622
training accuracy = 0.31, training loss = 0.644722
training accuracy = 0.17, training loss = 0.636620
Epoch 4: test accuracy = 0.234375
training accuracy = 0.14, training loss = 0.635601
training accur

In [300]:
# Training within each subject

#sub_num = [1, 2, 3, 4, 5, 6, 7, 8, 9]
sub_num = [1]

display_step_sub = 3

for i in sub_num:

    with tf.Session() as session:
        tf.global_variables_initializer().run()
        for epoch in range(training_epochs_sub):
            for b in range(total_batches_sub):    
                offset = (b * batch_size_sub) % (y_train_sub[i-1].shape[0] - batch_size_sub)
                batch_x = X_train_sub[i-1][offset:(offset + batch_size_sub), :]
                batch_y = y_train_sub[i-1][offset:(offset + batch_size_sub), :]
                batch_y_steps = np.tile(batch_y,((X_train_sub[i-1].shape[1]),1))
                
                _, acc, loss = session.run([optimizer, accuracy, loss_function], feed_dict={
                    x: batch_x, y: batch_y, y_steps: batch_y_steps, dropout: 0.3})
                
                if (b % display_step_sub == 0):
                    print('For subject %d : training accuracy = %.2f, training loss = %f' % (i, acc, loss))

            test_accuracy = session.run(accuracy, feed_dict={
                x:X_test_sub[i-1], y:y_test_sub[i-1], y_steps: y_test_steps_sub[i-1], dropout: 0})
            print('For subject %d, Epoch %d: test accuracy = %f' % (i, epoch, test_accuracy))

For subject 1 : training accuracy = 0.13, training loss = 1.653416
For subject 1 : training accuracy = 0.33, training loss = 1.608356
For subject 1 : training accuracy = 0.40, training loss = 1.557815
For subject 1 : training accuracy = 0.27, training loss = 1.546753
For subject 1 : training accuracy = 0.40, training loss = 1.502753
For subject 1, Epoch 0: test accuracy = 0.220000
For subject 1 : training accuracy = 0.40, training loss = 1.463342
For subject 1 : training accuracy = 0.33, training loss = 1.439098
For subject 1 : training accuracy = 0.27, training loss = 1.389731
For subject 1 : training accuracy = 0.13, training loss = 1.370708
For subject 1 : training accuracy = 0.20, training loss = 1.339989
For subject 1, Epoch 1: test accuracy = 0.240000
For subject 1 : training accuracy = 0.27, training loss = 1.312333
For subject 1 : training accuracy = 0.40, training loss = 1.248978
For subject 1 : training accuracy = 0.33, training loss = 1.220861
For subject 1 : training accura